# Dependences

In [2]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")

nearest_neighbors (generic function with 1 method)

# Model

In [3]:
@make_struct_func Cubic
@make_struct_func LennardJones

@time Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 10000.0,
        dt    = 0.5,
        nₖₙₙ  = 50,
        nₛₐᵥₑ = 100
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input = "../../data/init/Sphere"
    )
)
dump(Model)

  0.443035 seconds (548.44 k allocations: 34.783 MiB, 2.83% gc time, 88.72% compilation time)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 10000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 50
    nₛₐᵥₑ: Int64 100
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"
  ForceType: Array{Any}((2, 2))
    1: Int64 1
    2: Int64 2
    3: UnionAll
      var: TypeVar
        name: Symbol T
        lb: Union{}
        ub: Any
      body: Cubic{T} <: ForceType
        μ₁::T
        rₘᵢₙ::T
        rₘₐₓ::T
    4: UnionAll
      var: TypeVar
        name: Symbol T
        lb: Union{}
        ub: Any
      body: LennardJones{T} <: ForceType
        μ₁::T
        rₘᵢₙ::T
        rₘₐₓ::T


# Aggregate

In [4]:
FusionAGG = Nothing
@time FusionAGG = FusionAggregate(
    [
        AggType(
            "HEK_1", 
            InteractionPar(
                Cubic(1.0,2.0,3.0), 
                ContractilePar(0.01)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        ),
        AggType(
            "HEK_2", 
            InteractionPar(
                Cubic(1.0,2.0,3.5), 
                ContractilePar(0.02)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    Model
)
show_aggregates(FusionAGG)

  7.526430 seconds (15.44 M allocations: 1.270 GiB, 3.68% gc time, 4.47% compilation time)
========================= Type =======================


2-element Vector{AggType}:
 AggType("HEK_1", InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar(0.01)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})
 AggType("HEK_2", InteractionPar(Cubic{Float64}(1.0, 2.0, 3.5), ContractilePar(0.02)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

======================   Index =======================
Index of List of Aggregates


1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  2  2  2  2  2  2  2  2  2  2  2  2

Index of Number of Aggregates


...

Contractile


2×1 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.01
 0.02

Radius


2-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27
 15.27

------------------ Neighbors Size -------------------
idx      = (5008, 5008)
idx_red  = (39, 5008)
idx_sum  = (1, 5008)
idx_cont = (50, 5008)
------------------- Forces Size ---------------------
dX       = (5008, 3)
F        = (5008, 3)


# kNN

In [5]:
println("------------------------ SIZE ------------------------")
println("r_max         = $(FusionAGG.Type[1].Interaction.Force.rₘₐₓ)")
println("Size X        = $(size(FusionAGG.Position))")
println("Size idx      = $(size(FusionAGG.Simulation.Neighbor.idx))")
println("Size idx_real  = $(size(FusionAGG.Simulation.Neighbor.idx))")
println("Size idx_sum   = $(size(FusionAGG.Simulation.Neighbor.idx_sum))")
println("Size idx_red   = $(size(FusionAGG.Simulation.Neighbor.idx_red))")
@time nearest_neighbors(FusionAGG)
println("------------------------ IDX -------------------------")
println("idx")
display(FusionAGG.Simulation.Neighbor.idx)
println("idx_red")
display(FusionAGG.Simulation.Neighbor.idx_red)
println("idx_sum")
display(FusionAGG.Simulation.Neighbor.idx_sum)
println("idx_cont")
display(FusionAGG.Simulation.Neighbor.idx_cont)

------------------------ SIZE ------------------------
r_max         = 3.0
Size X        = (5008, 3)
Size idx      = (5008, 5008)
Size idx_real  = (5008, 5008)
Size idx_sum   = (1, 5008)
Size idx_red   = (39, 5008)
Threads = (32, 32) | Blocks  = (157, 157)
Threads = 256 | Blocks  = 20
Threads = (32, 32) | Blocks  = (157, 157)
 20.874853 seconds (31.21 M allocations: 1.773 GiB, 2.94% gc time, 17.35% compilation time)
------------------------ IDX -------------------------
idx


5008×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  0   0   1   1   0   0   0   0  …     0     0     0     0     0     0
 2  2  2   0   0   2   2   0   0   0        0     0     0     0     0     0
 0  3  3   0   0   0   3   3   0   0        0     0     0     0     0     0
 0  0  0   4   4   0   0   0   4   4        0     0     0     0     0     0
 5  0  0   5   5   5   0   0   0   5        0     0     0     0     0     0
 6  6  0   0   6   6   6   0   0   0  …     0     0     0     0     0     0
 0  7  7   0   0   7   7   7   0   0        0     0     0     0     0     0
 0  0  8   0   0   0   8   8   0   0        0     0     0     0     0     0
 0  0  0   9   0   0   0   0   9   9        0     0     0     0     0     0
 0  0  0  10  10   0   0   0  10  10        0     0     0     0     0     0
 0  0  0   0  11  11   0   0   0  11  …     0     0     0     0     0     0
 0  0  0   0   0  12  12   0   0   0        0     0     0     0     0     0
 0  0  0   0   0   0  13  13   0   0

idx_red


39×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
  1   1   2   4   1   1   2   3   4  …  4877  4878  4879  4887  4888  4889
  2   2   3   5   4   2   3   7   9     4948  4949  4950  4956  4957  4958
  5   3   7   9   5   5   6   8  10     4949  4950  4951  4957  4958  4959
  6   6   8  10   6   6   7  13  15     4950  4951  4952  4958  4959  4960
 35   7  37  40  10   7   8  14  48     4956  4957  4958  4963  4964  4965
 36  36  38  41  11  11  12  44  49  …  4957  4958  4959  4964  4965  4966
 37  37  39  42  41  12  13  45  50     4958  4959  4960  4965  4966  4967
 42  38  44  49  42  42  43  46  57     4959  4960  4961  4966  4967  4968
 43  43  45  50  43  43  44  53  58     4964  4965  4966  4971  4972  4973
 51  44  53  58  50  44  45  54  66     4965  4966  4967  4972  4973  4974
  0  52   0   0  51  51  52  62   0  …  4966  4967  4968  4973  4974  4975
  0   0   0   0  59  52  53   0   0     4972  4973  4974  4976  4977  4978
  0   0   0   0   0  60  61   0   0     4973  4974

idx_sum


1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 10  11  10  10  12  13  13  11  10  …  21  20  23  24  24  21  20  21  19

idx_cont


50×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 37  43  44  10  50   7   7  14   4  …  5004  5006  4961  4956  4967  4959
 51   1   8  10  10   5  43  44  10     4949  5005  4958  4972  5002  4960
  2  52  44  41   1   5  43  46  50     4997  5005  4999  5002  5003  4975
  6  52   2  42   4  11  44   8   9     5002  4973  4950  4887  5007  4968
 51   6  44  10  51  44  44  62  48     5003  4950  5005  4958  4974  4978
  2  36  45  40  51  60  43  62  10  …  4877  4974  4975  5003  4958  4973
  6  44   7  49  51   6  44  44  48     5003  4998  4968  4973  4967  4967
  5   3   7   4  42  60  12   3  66     4964  4958  4967  5003  4972  4978
  5  36  38  42  11  60  45   7  49     4959  4966  4974  4972  4973  4889
 37  38   3   5  42  42   3   8  50     4958  5003  4967  5006  4964  4965
  1  43  38  41  50  43  53  54  58  …  4999  4966  4999  4964  5008  4979
  1   3  45   4  11  11  45   7  49     4958  5000  4961  4977  4957  4966
 36  38  37  42   4  52  61   8  49     4973  5000

# Force

In [6]:
println("rₘₐₓ")
display(FusionAGG.Simulation.Parameter.rₘₐₓ)
println("Force")
display(FusionAGG.Simulation.Parameter.Force)
println("Contractile")
display(FusionAGG.Simulation.Parameter.Contractile)
println("Radius")
display(FusionAGG.Simulation.Parameter.Radius)

rₘₐₓ


2-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 3.0
 3.5

Force


2×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 1.0  2.0  3.0
 1.0  2.0  3.5

Contractile


2×1 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.01
 0.02

Radius


2-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27
 15.27

In [87]:
forc_f(type, param, dist) = force_func(type(param...),dist)
function sum_force!(idx,idx_contractile,points,force,force_idx,A_con,t_knn,Param)
    # Defining Index for kernel
    i = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    k = (blockIdx().y - 1) * blockDim().y + threadIdx().y

    # Limiting data inside matrix
    if i <= size(points, 1) && k <= size(points, 2)

        # Cleaning idx_sum
        force[i,k] = 0
        # dist = 0

        # Iterate on each row
        for j=1:size(idx,1)

            # Finding forces
            if idx[j,i] != i && idx[j,i] != 0
                dist = euclidean(points,i,idx[j,i])
                # force[i,k] +=force_func(forcetype(force_idx,Param...),dist) * (points[i,k]-points[idx[j,i],k]) / dist
                force[i,k] += force_func(Cubic(1.0,2.0,3.0),dist)
                
            end
            
        end

        # # Adding Contrractile Force
        # if idx_contractile[t_knn,i] != i && idx_contractile[t_knn,i] != 0
        #     dist = euclidean(points,i,idx_contractile[t_knn,i])
        #     force[i,k] += A_con*(points[i,k]-points[idx_contractile[t_knn,i],k])/dist
        # end
        
    end
    return nothing
end

sum_force! (generic function with 3 methods)

In [38]:
Model.ForceType[1,2]

Cubic

In [23]:
force_func(Cubic(1.0,2.0,3.0),2.5)

-0.125

In [81]:
forc_f(Cubic, Param, 2.5)

-0.125

In [25]:
idx_red         = FusionAGG.Simulation.Neighbor.idx_red
idx_contractile = FusionAGG.Simulation.Neighbor.idx_cont
points          = FusionAGG.Position
force           = FusionAGG.Simulation.Force.F

Param   = FusionAGG.Simulation.Parameter.Force[1,:]
A_con   = FusionAGG.Simulation.Parameter.Contractile[1,:]
t_knn = 1
force_idx = FusionAGG.Index.ForceType[1]

1

In [93]:
a,b,c = Param

3-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 1.0
 2.0
 3.0

In [96]:
c

3.0f0

In [79]:
Cubic(Param...)

Cubic{Float32}(1.0f0, 2.0f0, 3.0f0)

In [58]:
force_type = Model.ForceType[:,2]

2-element Vector{Any}:
 Cubic
 LennardJones

In [60]:
force_type[2]

LennardJones

In [88]:
# GPU
threads =(100,3)
blocks  =(cld.(size(points,1)+1,threads[1]),1)

@cuda threads=threads blocks=blocks sum_force!(idx_red,idx_contractile,points,force,force_idx,A_con,Int(t_knn),Param)
display(force)

5008×3 CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -0.063209   -0.063209   -0.063209
 -0.063209   -0.063209   -0.063209
 -0.063209   -0.063209   -0.063209
 -0.0803607  -0.0803607  -0.0803607
 -0.0785784  -0.0785784  -0.0785784
 -0.0767962  -0.0767962  -0.0767962
 -0.0767962  -0.0767962  -0.0767962
 -0.0785784  -0.0785784  -0.0785784
 -0.0718151  -0.0718151  -0.0718151
 -0.0768267  -0.0768267  -0.0768267
 -0.0768267  -0.0768267  -0.0768267
 -0.0768267  -0.0768267  -0.0768267
 -0.0768267  -0.0768267  -0.0768267
  ⋮                      
 -3.93797    -3.93797    -3.93797
 -3.93797    -3.93797    -3.93797
 -3.61631    -3.61631    -3.61631
 -2.97105    -2.97105    -2.97105
 -2.99705    -2.99705    -2.99705
 -3.31693    -3.31693    -3.31693
 -3.31515    -3.31515    -3.31515
 -3.31515    -3.31515    -3.31515
 -2.99527    -2.99527    -2.99527
 -2.998      -2.998      -2.998
 -2.998      -2.998      -2.998
 -2.68426    -2.68426    -2.68426

PENDING

# review

## force_func

In [ ]:
# Param = [1.0,2.0,4.0]

In [ ]:
# force_func(forcetype(1,Param...),3.0)

In [ ]:
# force_func(forcetype(2,Param...),3.0)